<a href="https://colab.research.google.com/github/wendy60/Hybrid-recommender-system/blob/initial-user-interest-model/NPA_Neural_News_Recommendation_with_Personalized_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scrapbook

In [ ]:
pip install recommenders

In [ ]:
pip install tensorflow-gpu==1.15.2

     |████████████████████████████████| 410.9 MB 34 kB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 503 kB 61.7 MB/s 
     |████████████████████████████████| 3.8 MB 29.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=db5153b90b6cbc1922e3b7c283c4a5fc143f941f1b15409ed5f50b74e3c3010f
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.npa import NPAModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


In [ ]:
epochs = 5
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'

In [ ]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'npa.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)


100%|██████████| 17.0k/17.0k [00:01<00:00, 12.0kKB/s]
100%|██████████| 9.84k/9.84k [00:00<00:00, 11.4kKB/s]
100%|██████████| 95.0k/95.0k [00:03<00:00, 23.9kKB/s]


In [ ]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs)
print(hparams)

data_format=news,iterator_type=None,support_quick_scoring=False,wordEmb_file=/tmp/tmp_0rk690j/utils/embedding.npy,wordDict_file=/tmp/tmp_0rk690j/utils/word_dict.pkl,userDict_file=/tmp/tmp_0rk690j/utils/uid2index.pkl,vertDict_file=None,subvertDict_file=None,title_size=10,body_size=None,word_emb_dim=300,word_size=None,user_num=None,vert_num=None,subvert_num=None,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=None,filter_num=400,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=100,learning_rate=0.0001,loss=cross_entropy_loss,optimizer=adam,epochs=5,batch_size=32,show_step=100000,metrics=['group_auc', 'mean_mrr', 'ndcg@5;10']


In [ ]:
iterator = MINDIterator

In [ ]:
model = NPAModel(hparams, iterator, seed=seed)

In [ ]:
print(model.run_eval(valid_news_file, valid_behaviors_file))


8874it [01:07, 130.74it/s]


{'group_auc': 0.5228, 'mean_mrr': 0.2328, 'ndcg@5': 0.2377, 'ndcg@10': 0.303}


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)


1086it [00:34, 31.57it/s]
8874it [01:03, 138.80it/s]


at epoch 1
train info: logloss loss:1.5022027583631583
eval info: group_auc:0.5924, mean_mrr:0.26, ndcg@10:0.3476, ndcg@5:0.2828
at epoch 1 , train time: 34.4 eval time: 71.4


1086it [00:30, 35.05it/s]
8874it [01:03, 139.20it/s]


at epoch 2
train info: logloss loss:1.4069186054979679
eval info: group_auc:0.6001, mean_mrr:0.2707, ndcg@10:0.3586, ndcg@5:0.2956
at epoch 2 , train time: 31.0 eval time: 71.4


1086it [00:31, 34.70it/s]
8874it [01:03, 139.04it/s]


at epoch 3
train info: logloss loss:1.3471880993131775
eval info: group_auc:0.5932, mean_mrr:0.2716, ndcg@10:0.3592, ndcg@5:0.2948
at epoch 3 , train time: 31.3 eval time: 71.3


1086it [00:30, 35.09it/s]
8874it [01:03, 140.08it/s]


at epoch 4
train info: logloss loss:1.3048143307807036
eval info: group_auc:0.5887, mean_mrr:0.2676, ndcg@10:0.3532, ndcg@5:0.2887
at epoch 4 , train time: 31.0 eval time: 70.9


1086it [00:30, 35.13it/s]
8874it [01:03, 139.53it/s]


at epoch 5
train info: logloss loss:1.2641648422947247
eval info: group_auc:0.6001, mean_mrr:0.2716, ndcg@10:0.3599, ndcg@5:0.2941
at epoch 5 , train time: 30.9 eval time: 71.0
CPU times: user 7min 8s, sys: 24.5 s, total: 7min 32s
Wall time: 8min 34s


In [ ]:

%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

8874it [01:03, 139.64it/s]


{'group_auc': 0.6001, 'mean_mrr': 0.2716, 'ndcg@5': 0.2941, 'ndcg@10': 0.3599}
CPU times: user 56.5 s, sys: 4 s, total: 1min
Wall time: 1min 11s


In [ ]:

sb.glue("res_syn", res_syn)

In [ ]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "npa_ckpt"))

In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_slow_eval(valid_news_file, valid_behaviors_file)


8874it [01:03, 139.49it/s]


In [ ]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')


7538it [00:00, 34619.64it/s]


In [ ]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()